In [42]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/'
#xml_dir = '/media/yyp/DATA/Public_Datasets/COCO/2017/Annotations/format_voc/Annotations/'
#xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/test/'
#image_dir = '/data/darknet/python/' + time_str + '_image/'
#generated_image_dir = image_dir
#generated_xml_dir = xml_dir

#generated_image_dir = '/data/darknet/python/' + time_str + '_image_rider/'
generated_xml_dir = xml_dir

os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + generated_xml_dir)

SEED_XML_DIR = xml_dir
GENE_XML_DIR = generated_xml_dir


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_bbox_from_xmlobj(obj_element_):
    xmlbox = obj_element_.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]

def get_obj_from_xml(xml_name):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    for obj in root.iter('object'):
        print(obj)
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    print(img_obj.shape)
    return img_obj
def xml_reverse_xmin_xmax(etree):
    root = etree.getroot()
    objects = root.findall('object')
    for obj in objects:
        xmlbox = obj.find('bndbox')
        xmlbox.find('xmin').text, xmlbox.find('xmax').text = xmlbox.find('xmax').text, xmlbox.find('xmin').text
    return etree
        
def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object

def generate_new_xmlobj_with_given_class(cls_name, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = cls_name

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = '0'

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position

def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
def generate_union_bbox(matrix1, matrix2): #xin,xmax,ymin,ymax
    img_obj_shape = (int(max(matrix1[3], matrix2[3])-min(matrix1[2], matrix2[2])), 
                int(max(matrix1[1], matrix2[1])-min(matrix1[0], matrix2[0])))
    return img_obj_shape

def generate_union_bbox_shape(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    img_obj_shape = (int(max(matrix_array[:,3]) - min(matrix_array[:,2])),
                     int(max(matrix_array[:,1]) - min(matrix_array[:,0])))
    return img_obj_shape

def get_union_bbox_left_top(matrix_list): #xin,xmax,ymin,ymax
    matrix_array = np.array(matrix_list)
    left_top = (int(min(matrix_array[:,0])),
                     int(min(matrix_array[:,2])))
    return left_top
def generate_union_bbox_(matrix_list):
    img_obj_shape = generate_union_bbox_shape(matrix_list)
    left_top = get_union_bbox_left_top(matrix_list)
    union_bbox = [int(left_top[0]), 
                  int(left_top[0]+int(img_obj_shape[1])),
                  int(left_top[1]),
                  int(left_top[1]+int(img_obj_shape[0]))]
    return union_bbox

def generate_new_bbox_within_distance(img_size, img_obj_size, old_bbox, scale=2.5):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    #array_y = np.arange(int(img_size[0] - img_obj_size[0])/3, int(img_size[0] - img_obj_size[0])/3*2)
    array_y = np.arange(max(0, int((old_bbox[3]+old_bbox[2])/2 - scale*img_obj_size[0])),
                        min(int((old_bbox[3]+old_bbox[2])/2 + scale*img_obj_size[0]), int(img_size[0] - img_obj_size[0])))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox

def check_before_insert(img_size, img_obj_size, etree, union_bbox):
    new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 100):
            return (False, False)
        new_bbox = generate_new_bbox_within_distance(img_size, img_obj_size, union_bbox)
    return (new_bbox[0], new_bbox[2])
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def do_inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree, insert_point):
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, insert_point, obj_element, etree)    
    return new_pil_img, new_xml_etree

def inset_dobule_obj_to_an_image_and_xml(img_obj1, img_obj2, img, obj_element1, obj_element2, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    retry_times = 0
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        retry_times = retry_times + 1
        if(retry_times > 50):
            return False, False
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, etree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0):
            return False
    return True
def caculate_move_vector(bbox, new_point):
    original_xmin, original_ymin = bbox[0], bbox[2]
    diffx, diffy = new_point[0] - original_xmin, new_point[1] - original_ymin
    return (diffx, diffy)

def rotate_img(img, thealta):
    (h_, w_) = img.shape[:2]
    point_list = [(0, 0), (0, h_), (w_, h_), (w_, 0)]
    center = (w_ // 2, h_ // 2)
    roted_point_list = [((point[0]-center[0])*np.cos(np.pi*thealta/180) 
                         - (point[1]-center[1])*np.sin(np.pi*thealta/180) 
                         + center[0], (point[0]-center[0])*np.sin(np.pi*thealta/180) 
                         + (point[1]-center[1])*np.cos(np.pi*thealta/180) + center[1]) for point in point_list]
    #print(roted_point_list)

    temp = np.zeros((2, 4))
    temp[0] = [roted_point[0] for roted_point in roted_point_list]
    temp[1] = [roted_point[1] for roted_point in roted_point_list]
    (xmin, xmax, ymin, ymax) = (np.min(temp[0]), np.max(temp[0]), np.min(temp[1]), np.max(temp[1]))
    print('xmin: {0}, xmax: {1}, ymin: {2}, ymax: {3}'.format(xmin, xmax, ymin, ymax))

    roted_h, roted_w = ymax - ymin, xmax - xmin
    print('roted_h: {0}, roted_w: {1}'.format(roted_w, roted_h))
    
    top_bottom, left_right = int((roted_h - h_)/2), int((roted_w - w_)/2)
    print('top_bottom:{0}, left_right_:{1}'.format(top_bottom, left_right))
    padding = lambda arg : max(arg, 0)
    #dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
    #padding(left_right), cv2.BORDER_CONSTANT)
    dst = cv2.copyMakeBorder(img, padding(top_bottom), padding(top_bottom), padding(left_right), 
                             padding(left_right), cv2.BORDER_CONSTANT)
    
    (h, w) = dst.shape[:2]   
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, thealta, 1.0)
    rotated_img = cv2.warpAffine(dst, M, (w, h))
    if(top_bottom < 0):
        top_bottom = abs(top_bottom)
        rotated_img = rotated_img[top_bottom:-top_bottom, :, :]
    if(left_right < 0):
        left_right = abs(left_right)
        rotated_img = rotated_img[:, left_right:-left_right, :]
    return rotated_img

In [43]:
################################# for motorbike-person pairs  ######################################
LOOP_TIMES = 1
IOU_THRESH = 0.1
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

for i in range (LOOP_TIMES):
    for xml_name in seed_xml_names:
        if(not xml_name.endswith('.xml')):
            continue
        xml_name = SEED_XML_DIR + xml_name
        #print(xml_name)
                
        tree=ET.parse(xml_name)
        root = tree.getroot()
        #img = cv2.imread(img_name)
        #objs = root.findall('object')
        bboxes = get_bboxes_from_etree(tree)
        if(bboxes[0][0] < bboxes[0][1]):
            continue
        print(xml_name)
        ##fix the wrong xml witch mixed the xmin and xmax
        tree = xml_reverse_xmin_xmax(tree)
        bboxes = get_bboxes_from_etree(tree)        
        
        tree.write(SEED_XML_DIR  + xml_name.split('/')[-1])
        continue
             
        flag_found = False
        i = 0
        #------------------------------------------------------------
        bboxes_motorbike = {}
        bboxes_person = {}
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            #print(bbox)
            cls = get_cls_from_xmlobj(obj_element)
            if (cls == 'motorbike' or cls == 'bicycle'):
                bboxes_motorbike[(i)] = bbox
            if (cls == 'person'):
                bboxes_person[(i)] = bbox
            i = i + 1
        #caculate iou
        #print(bboxes_motorbike) 
        #print(bboxes_person)
        id_dic = {}
        for key, val in bboxes_motorbike.items():
            id_list = []
            for k, v in bboxes_person.items():
                if(IOU(val, v) >= IOU_THRESH):
                    id_list.append(k)
            id_dic[(key)] = id_list
        print(id_dic)
        REPEAT_TIMES = 1
        FLAG = False
        for i in range(REPEAT_TIMES):
            for kk, vv in id_dic.items():
                if(len(vv) != 1):
                    continue
                else:
                    #print(bboxes_motorbike[kk])
                    #print(bboxes[kk])
                    #print(bboxes_person[vv[0]])
                    #print(bboxes[vv[0]])
                    img_obj_shape = generate_union_bbox(bboxes[kk], bboxes[vv[0]])
                    union_bbox_letf_top = (int(min(bboxes[kk][0], bboxes[vv[0]][0])), 
                                           int(min(bboxes[kk][2], bboxes[vv[0]][2])))
                    union_bbox = [int(min(bboxes[kk][0], bboxes[vv[0]][0])), 
                                  int(min(bboxes[kk][0], bboxes[vv[0]][0]))+int(img_obj_shape[1]),
                                  int(min(bboxes[kk][2], bboxes[vv[0]][2])),
                                  int(min(bboxes[kk][2], bboxes[vv[0]][2]))+int(img_obj_shape[0])]
                    
                    ## to keep original person and motorbike bicycle , do not remove them
                    
                    temp_objs_array = root.findall('object')
                    if(objs[kk] in temp_objs_array): 
                        root.remove(objs[kk])
                    temp_objs_array = root.findall('object')
                    if(objs[vv[0]] in temp_objs_array):
                        root.remove(objs[vv[0]])
                    
                    
                    cls_name = get_cls_from_xmlobj(objs[kk])
                    cls_name = 'rider-' + cls_name[:2]
                    
                    new_xml_obj = generate_new_xmlobj_with_given_class(cls_name, union_bbox_letf_top, img_obj_shape)
                    root.append(new_xml_obj)
                    FLAG = True
            #break                
            #------------------------------------------------------------
        ## save xml and img
        if(FLAG != True):
            continue
        #time_mark = datetime.datetime.now()
        #time_str = time_mark.strftime("%Y%m%d%H%M%S%f_")
        tree.write(GENE_XML_DIR  + xml_name.split('/')[-1])
        #break
        

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522910372_00002100000.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522920949_00002100001.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522930998_00002100002.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522940538_00002100003.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522949491_00002100004.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522957464_00002100005.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145522966151_00002100006.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/2019032814552

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523523667_00002100077.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523532314_00002100078.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523541066_00002100079.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523549820_00002100080.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523557721_00002100081.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523565422_00002100082.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145523572949_00002100083.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/2019032814552

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524114082_00002100146.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524123296_00002100147.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524133041_00002100148.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524141470_00002100149.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524150817_00002100150.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524158929_00002100151.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524168184_00002100152.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/2019032814552

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524700081_00002100213.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524709118_00002100214.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524718251_00002100215.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524726783_00002100216.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524736874_00002100217.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524745271_00002100218.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145524754174_00002100219.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/2019032814552

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145525792461_00002100313.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145525840304_00002100314.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145525886330_00002100315.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145525932576_00002100316.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145525979808_00002100317.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145526025765_00002100318.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328145526074229_00002100319.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/2019032814552

...

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925461796_ge_shrink_scale_0_0.3_127.5_00002100714.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925496317_ge_shrink_scale_0_0.3_127.5_00002100715.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925533791_ge_shrink_scale_0_0.3_127.5_00002100716.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925570205_ge_shrink_scale_0_0.3_127.5_00002100717.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925613134_ge_shrink_scale_0_0.3_127.5_00002100718.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151925648995_ge_shrink_scale_0_0.3_127.5_00002100719.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928554965_ge_shrink_scale_0_0.3_127.5_00002100770.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928591154_ge_shrink_scale_0_0.3_127.5_00002100771.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928633109_ge_shrink_scale_0_0.3_127.5_00002100772.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928694867_ge_shrink_scale_0_0.3_127.5_00002100773.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928750893_ge_shrink_scale_0_0.3_127.5_00002100774.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151928786112_ge_shrink_scale_0_0.3_127.5_00002100775.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931017668_ge_shrink_scale_0_0.3_127.5_00002100826.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931056158_ge_shrink_scale_0_0.3_127.5_00002100827.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931092815_ge_shrink_scale_0_0.3_127.5_00002100828.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931128129_ge_shrink_scale_0_0.3_127.5_00002100829.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931170264_ge_shrink_scale_0_0.3_127.5_00002100830.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151931207732_ge_shrink_scale_0_0.3_127.5_00002100831.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151934860423_ge_shrink_scale_0_0.3_127.5_00002100887.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151934911679_ge_shrink_scale_0_0.3_127.5_00002100888.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151934947538_ge_shrink_scale_0_0.3_127.5_00002100889.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151935001386_ge_shrink_scale_0_0.3_127.5_00002100890.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151935040556_ge_shrink_scale_0_0.3_127.5_00002100891.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151935077952_ge_shrink_scale_0_0.3_127.5_00002100892.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_

/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937256219_ge_shrink_scale_0_0.3_127.5_00002100942.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937312966_ge_shrink_scale_0_0.3_127.5_00002100943.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937350233_ge_shrink_scale_0_0.3_127.5_00002100944.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937388568_ge_shrink_scale_0_0.3_127.5_00002100945.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937424049_ge_shrink_scale_0_0.3_127.5_00002100946.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_Person/VOC2007/Annotations/20190328151937482843_ge_shrink_scale_0_0.3_127.5_00002100947.xml
/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_WuHan_